# Traitement des données
Pour plus d'information sur les sources de données, lisez le README.md

In [ ]:
import pandas as pd
import numpy as np


## 0. Traitement de la liste des communes

In [ ]:
# Télécharger et importer le fichier des communes au 1er janvier 2025 (à télécharger ici : https://www.insee.fr/fr/information/8377162)
df0 = pd.read_csv("data/raw_data/v_commune_2025.csv", sep=",")

# Ne garder que les communes seules (TYPECOM == 'COM')
df0 = df0[df0['TYPECOM'] == 'COM']

# Sélectionner les colonnes COM, LIBELLE
df0 = df0[['COM', 'LIBELLE']]

# Enregistrer le fichier nettoyé
df0.to_csv("data/processed/commune_2025_reduced.csv", index=False)

## 1. Traitement de RP (exploitation complémentaire) pour calculer les effectifs de CSP par commune

In [ ]:
# Télécharger et importer le fichier population et csp (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_RP_POPULATION_COMP "Télécharger la totalité du jeu de données")
df = pd.read_csv(r'data/raw_data/DS_RP_POPULATION_COMP_2022_data.csv', sep=';')

In [ ]:
# Sélection des communes uniquement, de l'année 2022, du sexe _T et de la tranche d'âge 'Y_GE15'
df1 = df[df['GEO_OBJECT'] == 'COM']
df1 = df1[df1['TIME_PERIOD'] == 2022]
df1 = df1[df1['SEX'] == '_T']
df1 = df1[df1['AGE'] == 'Y_GE15']

In [ ]:
# Drop des colonnes inutiles
df1 = df1.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'SEX', 'RP_MEASURE', 'AGE'])

In [ ]:
# Arrondis les valeurs de OBS à l'entier supérieur
df1["OBS_VALUE"] = np.ceil(df1["OBS_VALUE"])

In [ ]:
df1.head()

In [ ]:
# Exporter df1 en CSV
df1.to_csv('data/processed/population_communes_2022_csp.csv', index=False)

## 2. Traitement des données de l'INSEE sur la population municipale, le nombre de conseillers municipaux, la proportion de locataires et le taux de pauvreté

### 2.A. Population municipale et nombre de conseillers municipaux

In [ ]:
# Importer le fichier population municipale totale (population de référence) (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_POPULATIONS_REFERENCE "Télécharger la totalité du jeu de données")
df2 = pd.read_csv(r'data/raw_data/DS_POPULATIONS_REFERENCE_data.csv', sep=';')

# Sélectionner les communes, l'année 2022 et la population municipale (PMUN)
df2 = df2[df2['TIME_PERIOD'] == 2022]
df2 = df2[df2['GEO_OBJECT'] == 'COM']
df2 = df2[df2['POPREF_MEASURE'] == 'PMUN']

# Supprimer les colonnes GEO_OBJECT, TIME_PERIOD, POPREF_MEASURE et FREQ
df2 = df2.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'POPREF_MEASURE', 'FREQ'])

# Renommer la colonne OBS_VALUE en POPULATION_MUNICIPALE
df2 = df2.rename(columns={'OBS_VALUE': 'POPULATION_MUNICIPALE'})

In [ ]:

def calculer_conseillers_municipaux(fichier_repartition):
   """
   Calcule le nombre de conseillers municipaux pour chaque commune à partir du fichier de population municipale (df2) et du fichier "conseillers.csv" qui fournit la répartition selon les fourchettes de population.
   
   Parameter:
   -----------
   fichier_repartition : str
       Chemin vers le fichier CSV avec la répartition population/conseillers
       
   Returns:
   --------
   pandas.DataFrame
       DataFrame avec les communes et leur nombre de conseillers
   """
   
   # Charger le fichier
   repartition_df = pd.read_csv(fichier_repartition)
   
   # Charger le dataframe population
   communes_df = df2
   
   # Fonction pour déterminer le nombre de conseillers
   def get_nb_conseillers(population):
       for _, row in repartition_df.iterrows():
           if row['population_min'] <= population <= row['population_max']:
               return row['nombre_de_conseillers']
       return None
   
   # Calculer le nombre de conseillers pour chaque commune
   communes_df['NB_CONSEILLERS_TOTAL'] = communes_df['POPULATION_MUNICIPALE'].apply(get_nb_conseillers)
   
   communes_df.loc[communes_df['GEO'] == '75056', 'NB_CONSEILLERS_TOTAL'] = 163  # Paris
   communes_df.loc[communes_df['GEO'] == '69123', 'NB_CONSEILLERS_TOTAL'] = 73   # Lyon
   communes_df.loc[communes_df['GEO'] == '13055', 'NB_CONSEILLERS_TOTAL'] = 101  # Marseille

   return communes_df

In [ ]:
# Lancer la fonction avec le fichier de répartition
df2 = calculer_conseillers_municipaux('./data/conseillers.csv')

### 2.B. Traitement de RP (logement) pour calculer la proportion de locataires par commune

In [ ]:
# Importer le fichier logement principal (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_RP_LOGEMENT_PRINC "Télécharger la totalité du jeu de données")
df3 = pd.read_csv("data/raw_data/DS_RP_LOGEMENT_PRINC_2022_data.csv", sep=";")

# Ajouter des leads zeros aux codes INSEE pour qu'ils fassent 5 caractères
df3['GEO'] = df3['GEO'].apply(lambda x: str(x).zfill(5))


# Sélectionner les communes, l'année 2022, la mesure RP_MEASURE = 'DWELLINGS_POPSIZE', et L_STAY = '_T'
df3 = df3[df3['TIME_PERIOD'] == 2022]
df3 = df3[df3['GEO_OBJECT'] == 'COM']
df3 = df3[df3['RP_MEASURE'] == 'DWELLINGS_POPSIZE']
df3 = df3[df3['L_STAY'] == '_T']

# Supprimer les colonnes GEO_OBJECT, TIME_PERIOD, OCS, L_STAY, TDW, NRG_SRC, CARS, RP_MEASURE, CARPARK, NOR and BUILD_END
df3 = df3.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'OCS', 'L_STAY', 'TDW', 'NRG_SRC', 'CARS', 'RP_MEASURE', 'CARPARK', 'NOR', 'BUILD_END'])

# Round up the OBS_VALUE to the nearest integer
df3["OBS_VALUE"] = np.ceil(df3["OBS_VALUE"])

In [ ]:
# Pivoter le dataframe pour avoir une colonne par TSH
df3_pivot = df3.pivot_table(
    index='GEO',
    columns='TSH',
    values='OBS_VALUE',
    aggfunc='first'
).reset_index()

# # Catégories de locataires
categories_locataires = ['211', '212_222', '221', '300']

# # Calculer le total des locataires
df3_pivot['TOTAL_LOCATAIRES'] = df3_pivot[categories_locataires].sum(axis=1)

# Calculer la proportion de locataires
df3_pivot['PROPORTION_LOCATAIRES'] = df3_pivot.apply(
    lambda row: (row['TOTAL_LOCATAIRES'] / row['_T'] * 100) if row['_T'] > 0 else 0,
    axis=1
)

In [ ]:
# Fusionner df3_pivot avec df2 pour avoir les colonnes POPULATION_MUNICIPALE et NB_CONSEILLERS_TOTAL
# La fusion donne 34 922 communes (les 34 918 du fichier population municicipale (df2) + 4 communes supplémentaires présentes dans le fichier logement principal (df3)
# Ces 4 communes sont 12218 (ancienne commune), 14581 (mauvais code INSEE), 49126 (mauvais code INSEE), 69114 (ancienne commune)
df3 = pd.merge(df2, df3_pivot, on=['GEO'], how='outer')

# Calculer le nombre de conseillers en fonction de la proportion de locataires
# Formule : NB_CONSEILLERS_TOTAL * (PROPORTION_LOCATAIRES / 100)
df3['NB_CONSEILLERS_LOCATAIRES'] = (
    df3['NB_CONSEILLERS_TOTAL'] * (df3['PROPORTION_LOCATAIRES'] / 100)
).round(0).astype('Int64')  # Arrondir à l'entier et gérer les NaN

# Supprimer les colonnes inutiles 100, 200, 210, 211, 212_222, 221, 300
columns_to_drop = ['100', '200', '210', '211', '212_222', '221', '300']
df3 = df3.drop(columns=columns_to_drop, errors='ignore')

# Renommer la colonne _T en TOTAL_HABITANTS_FICHIER_LOGEMENT
df3 = df3.rename(columns={'_T': 'TOTAL_HABITANTS_FICHIER_LOGEMENT'})

In [ ]:
# Exporter df2 en CSV
df3.to_csv('data/processed/popmun_conseillers_locataires_2022.csv', index=False)